Notes:
2. try out torch.info, thop, weights and biases(wandb), comet.ml
3. integrate tensorboard, to track gradients and weights in histograms
4. move experiment folder out of work folder, make working dir one step back, make experiment folder seperate from what you will send to collab.
5. the experiment folder will have experiment dir, and trial notes. Trial notes will be the saved model, the pred list, and a markdown file (that has template) for notes after exmamining tensor board stats

In [ ]:
'''from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

for epoch in range(num_epochs):
    # Training loop here...
    # ...

    # Log weights to TensorBoard
    for name, param in model.named_parameters():
        writer.add_histogram(name, param, epoch)

writer.close()'''

In [2]:
# imports

import os
import torch
import numpy as np
import sklearn
import data_collection as dc
import model_handling as mh
import display as dis
import general_torch as gt
import models


In [ ]:
# starting new experiment, or do you have an old one
scratch_start = True
skinny_run = True
exp_dir = 'test'

In [ ]:
# init exp from scratch
if scratch_start:
    df = dc.initialize_dataframe()
    exp = dc.experiment_dict()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if device == "cuda" else torch.float
    exp['model'] = 'model0'
    exp['data_frame'] = df
    exp['optim'] = 'sgd'
    exp['optim_dict'] = {}  # if a value is part of the experiment, leave it empty and init it where appropriate
    exp['error_f'] = 'mse'
    exp['data_file'] = 'mnist.pkl'   # put the tensors into one dict and this will be the file extension!
    exp['dl_dict'] = {}  # 'data_batch_size': None, 'random_batch': False,
    exp['seed'] = 42
    exp['independent_var'] = 'lr'
    exp['device'] = device
    exp['data_type'] = dtype
else:
    exp = dc.load_experiment(exp_dir)
    df = exp['data_frame']

In [172]:
# raw data loader, change device, dtype and if needed shape
# this needs to change, take exp file, load the dict, use dict to distribute to tensors

dict_tensor = dc.load_tensor_dict(exp['data_file'])

train_data = dict_tensor['train_data'].to(exp['device']).to(exp['data_type'])
train_label = dict_tensor['train_label'].to(exp['device']).to(exp['data_type'])
test_data = dict_tensor['test_data'].to(exp['device']).to(exp['data_type'])
test_label = dict_tensor['test_label'].to(exp['device']).to(exp['data_type'])



In [228]:
# init data set

from torch.utils.data import Dataset

class Data(Dataset):
    def __init__(self, train, label):
        self.data = train
        self.label = label
      
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
        
train_set = Data(train_data, train_label)

In [ ]:
# always start by resetting seeds before initialzing anything
gt.init_seeds(exp['seed'], exp['device'])

In [229]:
# prep dataloader, use kwargs to init
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, **exp['dl_dict']) 

In [231]:
# build model / load state dict / load whole model / explicitly init param weights
import models

model = models.get_model(exp['model'])()
model.to(exp['data_type'])
model.to(exp['device'])


Model0_2(
  (lin1): Linear(in_features=1, out_features=50, bias=True)
  (lin2): Linear(in_features=50, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
# check for runtime errors / init predictions should be part of data collection loop


'''for i in model.parameters():
    print(i.dtype)
    break
print(test_data.dtype)'''

with torch.inference_mode():
    preds = model(test_data)
    
# show init preds / dict
model.state_dict()
dis.plot_multiclass_confusion_matrix(test_label.to('cpu').numpy(), preds.to('cpu').numpy(), ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [233]:
# set up hyperparameters, init with dict
switch = True
if switch:
    epochs = exp['epochs']
    optim = gt.get_optim(exp['optim'])(model.parameters(), **exp['optim_dict'])
    error = gt.get_error(exp['error_f'])()
else:
    epochs = 100
    optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.6)
    error = torch.nn.MSELoss()

In [234]:
# init tensorboard, use event monitor or profiler? skinny should use this

In [235]:
# need seperate data collection and examination loops here, also switch to dataloader, use skinny flag to fork collection
model.train()
if skinny_run:
    for _ in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            pred = model(data)
            '''for i in zip(train_pred, train_label):
                print(i)'''
            loss = error(pred, target)
            optim.zero_grad()
            loss.backward()
            optim.step()
else:
    # data collect loop
    pass

In [237]:
# run inference, greater analysis here, use data in eval?

with torch.inference_mode():
    preds = model(test_data)
    
# show init preds / dict
model.state_dict()
dis.plot_multiclass_confusion_matrix(test_label.to('cpu').numpy(), preds.to('cpu').numpy(), ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [ ]:
if skinny_run:
    pass
    # skiny eval with no data collection, but still with tensor board
else:
    pass
    # eval model -- tensorboard, events, profiler, take predictions overtime and create a seaborn graph, build a suite!

In [ ]:
# save model state dict, preds, and mkdown, need functions here

In [ ]:
#mkdown for notes